In [44]:
%load_ext autoreload
%autoreload 2
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
from scipy.spatial.distance import cosine as cosine_distance
import numpy as np
import sys
sys.path.append('../src')
from models import produce_effect_size

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
wv_from_bin = KeyedVectors.load_word2vec_format("../data/external/GoogleNews-vectors-negative300.bin", binary=True)  # C bin format

In [10]:
wv_from_bin['science']

array([ 0.0090332 ,  0.12158203,  0.3984375 ,  0.21386719, -0.01672363,
        0.06396484,  0.3125    , -0.23535156, -0.10009766, -0.09326172,
        0.20507812, -0.03222656, -0.21191406,  0.00958252, -0.27734375,
        0.09619141, -0.1875    , -0.04052734,  0.2265625 , -0.23632812,
       -0.04858398,  0.03295898, -0.08056641,  0.12255859,  0.05859375,
       -0.09130859, -0.02685547,  0.02587891, -0.11376953, -0.21972656,
       -0.0378418 , -0.15722656, -0.05981445, -0.01635742,  0.11425781,
       -0.15234375, -0.01086426,  0.29296875,  0.22265625, -0.00245667,
       -0.16308594, -0.11669922, -0.20800781,  0.31835938, -0.08740234,
        0.13378906,  0.15429688,  0.11816406, -0.07470703,  0.09326172,
       -0.32421875,  0.01239014, -0.07226562, -0.10986328, -0.07177734,
        0.16113281,  0.05541992, -0.3984375 , -0.04980469, -0.25195312,
       -0.05249023, -0.05786133,  0.16699219, -0.26953125, -0.04394531,
       -0.14257812,  0.03662109,  0.34765625, -0.10888672, -0.12

In [7]:
# WEAT 6
X_terms = ['John', 'Paul','Mike','Kevin','Steve','Greg','Jeff','Bill']
Y_terms = ['Amy','Joan','Lisa','Sarah','Diana','Kate','Ann','Donna']
A_terms = ['executive','management','professional','corporation',
               'salary','office','business','career']
B_terms = ['home','parents','children','family',
               'cousins','marriage','wedding','relatives']

In [15]:
# WEAT 7 using Google News word2vec model
X_terms = ['math','algebra','geometry','calculus',
             'equations','computation','numbers','addition']
Y_terms = ['poetry','art','dance','literature',
             'novel','symphony','drama','sculpture']
A_terms = ['male','man','boy','brother',
              'he','him','his','son']
B_terms = ['female','woman','girl','sister',
               'she','her','hers','daughter']

In [19]:
# Weat 8
A_terms = ['brother','father','uncle','grandfather',
          'son','he','his','him']
B_terms = ['sister','mother','aunt','grandmother',
          'daughter','she','hers','her']
X_terms = ['science','technology','physics','chemistry',
          'Einstein','NASA','experiment','astronomy']
Y_terms = ['poetry','art','Shakespeare','dance',
          'literature','novel','symphony','drama']

In [5]:
wv_test_object = {'x_word1':[.1,.2,.3],
            'x_word2':[.4,.5,.6],
            'y_word1':[.2,.4,.6],
            'y_word2':[.1,.3,.5],
            'a_word1':[.5,.7,.9],
            'a_word2':[.4,.6,.8],
            'b_word1':[.3,.6,.9],
            'b_word2':[.4,.7,.9]}
word_set = wv_test_object.keys()
word_set

dict_keys(['x_word1', 'x_word2', 'y_word1', 'y_word2', 'a_word1', 'a_word2', 'b_word1', 'b_word2'])

In [20]:
[X_mtx, Y_mtx, A_mtx, B_mtx] = [word_set_to_mtx(wv_from_bin, terms) for terms in [X_terms, Y_terms, A_terms, B_terms]]

In [11]:
X_mtx = word_set_to_mtx(wv_test_object, ['x_word1','x_word2'])
Y_mtx = word_set_to_mtx(wv_test_object, ['y_word1','y_word2'])
A_mtx = word_set_to_mtx(wv_test_object, ['a_word1','a_word2'])
B_mtx = word_set_to_mtx(wv_test_object, ['b_word1','b_word2'])
for m in [X_mtx, Y_mtx, A_mtx, B_mtx]:
    print(m)

NameError: name 'wv_test_object' is not defined

In [13]:
def calculate_association_metric_for_target_word(word_vec, A_mtx, B_mtx):
    '''Computes the association metric, s(w,A,B).
    word_vec: 1-D word vector
    A_mtx, B_mtx: 2-D word vector arrays'''
    A_cosines = np.apply_along_axis(lambda row: 1-cosine_distance(row, word_vec), 1, A_mtx)
    B_cosines = np.apply_along_axis(lambda row: 1-cosine_distance(row, word_vec), 1, B_mtx)
    return np.mean(A_cosines) - np.mean(B_cosines)

In [21]:
def calculate_effect_size(X_mtx, Y_mtx, A_mtx, B_mtx):
    '''Computes the effect size.
    X_mtx, Y_mtx, A_mtx, B_mtx: 2-D word vector arrays.'''
    x_associations = np.apply_along_axis(lambda x_vec: calculate_association_metric_for_target_word(x_vec, A_mtx, B_mtx), 1, X_mtx)
    y_associations = np.apply_along_axis(lambda y_vec: calculate_association_metric_for_target_word(y_vec, A_mtx, B_mtx), 1, Y_mtx)
    X_union_Y = np.vstack((X_mtx, Y_mtx))
    all_associations = np.apply_along_axis(lambda w_vec: calculate_association_metric_for_target_word(w_vec, A_mtx, B_mtx), 1, X_union_Y)
    return (np.mean(x_associations) - np.mean(y_associations))/np.std(all_associations, ddof=1)
calculate_effect_size(X_mtx, Y_mtx, A_mtx, B_mtx)

1.243854992736378

In [47]:
produce_effect_size(wv_from_bin, X_terms, Y_terms, A_terms, B_terms)

1.243854992736378

# Results
Weat 6: 1.88085
Weat 7: .88033
Weat 8: 1.523